In [9]:
import torch
from torch import nn
from d2l import torch as d2l

In [10]:

def num_gpus():  #@save
    """Return 1 if MPS (Apple Silicon GPU) is available, else 0."""
    return 1 if torch.backends.mps.is_available() else 0

def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu():  #@save
    """Get the MPS device if available, else fallback to CPU."""
    return torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

# Example usage
cpu(), gpu()


(device(type='cpu'), device(type='mps'))

In [16]:
def try_gpu():  #@save
    """Return MPS device if available, else CPU."""
    return gpu()

def try_all_gpus():  #@save
    """Return [MPS] if available, else [CPU]."""
    return [gpu()]

In [18]:
try_gpu(), try_all_gpus()

(device(type='mps'), [device(type='mps')])

In [19]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='mps:0')

In [22]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

/Users/vigneshvpai/miniforge3/envs/D2L/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [23]:
net(X)

tensor([[0.3258],
        [0.3258]], device='mps:0', grad_fn=<LinearBackward0>)

In [24]:
net[0].weight.data.device

device(type='mps', index=0)

In [26]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model